# Collecting Steam API
The data crawled from Steam Spy.

In [12]:
import pandas as pd
import numpy as np
import math, requests, json, re, time, sys, ast

from tqdm import tqdm

In [2]:
df = pd.read_csv('datasets/steam_dataset/steamapi/csv/steamapp_api.csv',sep=',')
df.head(2)

,appid,name,release_date,developer,publisher,is_free,initialprice,languages,genre,tags,detailed_description,about_the_game,short_description,header_image,website,pc_requirements,mac_requirements,linux_requirements
0,10,Counter-Strike,"{'coming_soon': False, 'date': '1 Nov, 2000'}",Valve,Valve,False,999,"English, French, German, Italian, Spanish - Sp...",Action,"{'Action': 5374, 'FPS': 4797, 'Multiplayer': 3...",Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,https://cdn.akamai.steamstatic.com/steam/apps/...,0,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual..."
1,10090,Call of Duty: World at War,"{'coming_soon': False, 'date': '18 Nov, 2008'}",Treyarch,Activision,False,1999,"English, French, Italian, German, Spanish - Spain",Action,"{'Zombies': 1108, 'World War II': 818, 'FPS': ...","Call of Duty is back, redefining war like you'...","Call of Duty is back, redefining war like you'...","Call of Duty is back, redefining war like you'...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.callofduty.com,"{'minimum': '<ul class=""bb_ul""><li><strong>Sup...",[],[]


In [14]:
x = ast.literal_eval(df['release_date'][0])
x['date']

'1 Nov, 2000'

## Requesting Steam Spy API - Game Summaries
Iterating through Steam Spy API pages to get the JSON files and convert it to CSV and Pandas DataFrame.

In [ ]:
# urls = []
# json_list = []

# print('Listing URLs...')
# for i in range(50):
#     urls.append(f'https://steamspy.com/api.php?request=all&page={i}')

# print('Done listing.\n----------------------')
# print('Loading JSON...')

# for url in urls:
#     data = requests.get(url)
#     if data.status_code != 200:
#         print(f'Error: {url}')
#         continue

#     json_list.append(data.json())
#     time.sleep(60)

# print('Loading successful!')

Dumping JSON to files.

In [ ]:
# with open('datasets/steam_dataset/json/steamapi_games.json', 'w+') as f:
#     json.dump(json_list, f, sort_keys=True, indent=None)

In [ ]:
# df = pd.DataFrame(json_list).to_csv('test.csv', encoding='utf-8', index=False, sep=";")

In [ ]:
# file = open('datasets/steam_dataset/steamapi/json/steamapp_api.json')
# steamapp_api = json.load(file)

In [ ]:
# list(steamapp_api[0].values())[0]

In [ ]:
# def get_json_values(api):
#     values = []
#     for i in range(len(api)):
#         for j in range(len(api[i])):
#             values.append(list(api[i].values())[j])
#     return values

In [ ]:
# json_values = get_json_values(steamapp_api)
# json_values

In [ ]:
# df_game_summaries = pd.DataFrame(json_values)
# df_game_summaries.head(3)

In [ ]:
# len(df_game_summaries)

In [ ]:
# df_game_summaries.to_csv('datasets/steam_dataset/steamapi/csv/game_summaries.csv')

## Requesting Steam Spy API - Game Details
Getting game details from Steam Spy API based on games id (```appid```).

In [ ]:
df_game_summaries = pd.read_csv('datasets/steam_dataset/steamapi/csv/game_summaries.csv', sep=',')

In [ ]:
list_of_appids = df_game_summaries['appid'].to_list()
list_of_appids[0]

In [ ]:
json_list_details = []

In [ ]:
def get_seconds(prev, duration):
    next = prev + duration
    return next

In [ ]:
32400 + 3600

In [ ]:
prev = 32400 # nanti berarti prev -> 36000
next = get_seconds(prev, 10)

In [ ]:
urls_details = []
failed_request_id = []

print('Listing URLs...')
for i in range(prev, next):
    urls_details.append(f'https://steamspy.com/api.php?request=appdetails&appid={list_of_appids[i]}')

print('Done listing.\n----------------------')
print('Loading JSON...')

for url in tqdm(urls_details):
    data = requests.get(url)
    if data.status_code != 200:
        print(f'Error: {url}')
        failed_request_id.append(url)
        continue

    json_list_details.append(data.json())
    time.sleep(1)

print('Loading successful!')

Create dataframe of JSON list details and convert into CSV file.

In [ ]:
df_game_details = pd.DataFrame(json_list_details)
df_game_details.head(3)

In [ ]:
len(df_game_details)

In [ ]:
df_game_details.head(3)

In [ ]:
df_game_details.to_csv('datasets/steam_dataset/steamapi/csv/game_details.csv')

Extending JSON list into one JSON file.

In [ ]:
with open('datasets/steam_dataset/steamapi/json/steamapp_details.json', "r") as file:
    game_details = json.load(file)

json_drop_duplicates = {each['appid'] : each for each in game_details}.values()
game_details.extend(json_list_details) # di mana data = json list dari hasil crawling.

Writing into a JSON file.

In [ ]:
with open('datasets/steam_dataset/steamapi/json/steamapp_details.json', 'w+') as file:
    json.dump(game_details, file, sort_keys=True, indent=None)

In [ ]:
len(game_details)

In [ ]:
df_game_summaries = pd.read_csv('datasets/steam_dataset/steamapi/csv/game_summaries.csv', sep=',')
with open('datasets/steam_dataset/steamapi/json/steamapp_details.json', "r") as file:
    json_file = json.load(file)
json_drop_duplicates = {each['appid'] : each for each in json_file}.values()

df_json = pd.DataFrame(json_drop_duplicates).astype(str)

In [ ]:
len(df_game_summaries)

In [ ]:
len(df_json)

In [ ]:
df_json['appid'] = df_json['appid'].astype(np.int64)

In [ ]:
df_excluded = df_game_summaries[~df_game_summaries['appid'].duplicated()]

In [ ]:
len(df_excluded)

In [ ]:
appid_summaries = set(df_excluded['appid'].to_list())
appid_json = set(df_json['appid'].to_list())

In [ ]:
diff = appid_summaries - appid_json

In [ ]:
len(diff)

In [ ]:
len(df_json) + len(diff)

In [ ]:
len(df_json)

In [2]:
x = [123, 122, 124, 125, 126]
print(x[3:5])

[125, 126]
